In [92]:
!{__import__('sys').executable} -m pip install --quiet --upgrade -r requirements.txt

In [4]:
session_storage = 'downloads'

import os
os.makedirs(session_storage, exist_ok=True)

In [5]:
import fsspec

fs_dataset = fsspec.filesystem(
    'simplecache', 
    target_protocol='https', 
    cache_storage=session_storage,
)

In [6]:
import pandas as pd
'''
import swifter
swifter.set_defaults(
    npartitions=None,
    dask_thres=1,
    scheduler='processes',
    progress_bar=True,
    progress_bar_desc=None,
    allow_dask_on_strings=True,
    force_parallel=False
)
'''

import sklearn as skl
import sklearn.base
import sklearn.compose
#import sklearn.ensemble
import sklearn.feature_extraction
import sklearn.linear_model
import sklearn.metrics
import sklearn.model_selection
import sklearn.pipeline
import sklearn.svm

import xgboost

import numpy as np

In [7]:
import sklearnex
sklearnex.patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## Dataset

See https://ftp.funet.fi/pub/mirrors/ftp.imdb.com/pub/frozendata

In [8]:
import re

def _parse_imdb_list(fo):
    pattern = re.compile(
        fr'^(?P<key>[^:]+):\s+(?P<value>.*)$', 
        flags=re.DOTALL
    )
    for l in fo.read().splitlines():
        m = re.match(pattern, l)
        if not m:
            continue
        yield m.groupdict()

def _parse_imdb_list_plot(f):
    r = None

    for entry in _parse_imdb_list(f):
        # 
        if entry['key'] == 'MV':
            if r is not None:
                yield r

            r = dict()
            r['name'] = entry['value']

        if r is None:
            # error
            continue

        #if entry['key'] == 'BY':
        #    r.setdefault('plot_authors', [])
        #    r['plot_authors'].append(entry['value'])

        if entry['key'] == 'PL':
            r.setdefault('plot_text', [])
            r['plot_text'].append(entry['value'])

In [9]:
dataset_plots = None
with fs_dataset.open(
    'https://ftp.funet.fi/pub/mirrors/ftp.imdb.com/pub/frozendata/plot.list.gz', 
    compression='gzip', 
    mode='rt', errors='ignore'
) as f:
    dataset_plots = pd.DataFrame(
        list(_parse_imdb_list_plot(f))
    )

In [10]:
dataset_plots

,name,plot_text
0,"""#7DaysLater"" (2013)",[#7dayslater is an interactive comedy series f...
1,"""#BlackLove"" (2015) {Bringing Sexy Back (#1.3)}","[This week, the five women work on getting wha..."
2,"""#BlackLove"" (2015) {Crash the Party (#1.9)}","[With just one week left in the workshops, the..."
3,"""#BlackLove"" (2015) {Like a Virgin (#1.4)}",[All of the women are struggling with what mak...
4,"""#BlackLove"" (2015) {Making Lemonade Out of Le...",[All of the women start making strides towards...
...,...,...
596421,deux (2017),[When 2 Musicians throw coins in the same wish...
596422,propos: philosophie (2016),[Is happiness good for human beings? Is truth ...
596423,a gote le ciel (2014),[The 'Sky's the Limit' is the story of an 11 y...
596424,solo un nastro che gira (2017),[A woman in crisis finds in a peculiar bar the...


In [11]:
dataset_plots.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 596426 entries, 0 to 596425
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   name       596426 non-null  object
 1   plot_text  596426 non-null  object
dtypes: object(2)
memory usage: 9.1+ MB


In [44]:
print(dataset_plots.loc[566200][1])

['Happy-go-lucky Malya flies through life without resistance, breezing along', 'on her charms, never questioning her relationship with her mother, her Down', 'Syndrome sister, or anyone else for that matter. But in one split-second', 'everything changes, sending Malya careening off in a different direction,', "until she's forced to stop herself and really think for the very first", "time. She wonders if life isn't a scale, and all the little things that we", 'do really do count. And maybe every word and every action is "Tipping the', 'Scale."']


In [66]:
from urllib.parse import urlparse
import os


os.path.split(
    urlparse('/Movie Scripts/Reservoir Dogs Script.html').path
)

('/Movie Scripts', 'Reservoir Dogs Script.html')

In [38]:
len(list(filter(lambda s: s.endswith('.html'), get_movie_links())))

1206

In [30]:
import requests
import lxml
import lxml.html

import os
import urllib


def get_all_hrefs(url):
    return map(
        lambda s: urllib.parse.urlparse(s).path,
        lxml.html.fromstring(
            requests.get(url).text
        ).xpath('//a/@href')
    )


import joblib

def get_script(script_link):
    res = ''
    for el in lxml.html.fromstring(
        requests.get('https://imsdb.com/' + script_link).text
    ).xpath('//td[@class="scrtext"]'):    
        res += ''.join(el.itertext())
    return res    
    
    
    
def get_movie_links():
    return set(
        filter(
            lambda u: os.path.dirname(u) == '/Movie Scripts',
            get_all_hrefs('https://imsdb.com/all-scripts.html')
        )
    )


    
def get_script_links(movie_link):
    return set(
        filter(
            lambda u: os.path.dirname(u) == '/scripts',            
            get_all_hrefs('https://imsdb.com/' + movie_link)
        )
    )
    
def get_movie_name(movie_link):
    name, _ = os.path.splitext(
        os.path.basename(movie_link)
    )
    return name
    

    
    
def get_movie_script_links():
    def _f(movie_link):
        return (
            get_movie_name(movie_link),
            get_script_links(movie_link)
        )
    return joblib.Parallel(n_jobs=-1)(
        joblib.delayed(_f)(movie_link) for movie_link in get_movie_links()
    )



def get_scripts(movie_script_links):
    def _f(movie_name, script_link):
        return (
            movie_name,
            list(map(
                get_script,
                script_link
            ))
        )


    return joblib.Parallel(n_jobs=-1)(
        joblib.delayed(_f)(movie_name, script_link) for movie_name, script_link in movie_script_links
    )



df = pd.DataFrame(
get_scripts(
    get_movie_script_links()
),
    columns=['script_name', 'script_text']
)




In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1209 entries, 0 to 1208
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   script_name  1209 non-null   object
 1   script_text  1209 non-null   object
dtypes: object(2)
memory usage: 19.0+ KB


In [39]:
df.to_pickle('./dataset.pkl', compression='gzip')

In [40]:
pd.read_pickle('./dataset.pkl', compression='gzip')

,script_name,script_text
0,Star Wars: The Force Awakens Script,[\r\n\r\n\r\n\r\n \r\n ...
1,Extract Script,[\r\n\r\n\r\n\r\n\r\n \r\n ...
2,Superfights Script,[]
3,Batman Returns Script,[]
4,Wonder Woman Script,[\r\n\r\n\r\n\r\n\r\n\r\n ...
...,...,...
1204,Un Singe en Hiver Script,[\r\n\r\n\r\n\r\n ...
1205,Frequency Script,[\r\n\r\n\r\n\r\n\r\n<b><!--\r\n</b>if (window...
1206,Source Code Script,[\r\n\r\n\r\n\r\n ...
1207,Harry Potter and the Chamber of Secrets Script,[]


In [29]:
#print(df.iloc[1]['script_text'][0])

In [9]:
import pandas as pd

In [3]:
str('Ace-Ventura-Pet-Detective.html').replace('.html', '')

'Ace-Ventura-Pet-Detective'

In [96]:
def get_movie_details(movie_links):
    pass



In [ ]:
script_links = list(script_links)



el = lxml.html.fromstring(
    requests.get('https://imsdb.com/' + script_links[0].path).text
).xpath('//td[@class="scrtext"]')

el_text = ''.join(el[0].itertext())

#print(el_text)

In [99]:
#url_movie_links

In [82]:
#url_movie_links

In [34]:
import requests
import lxml
import lxml.html


el = lxml.html.fromstring(
    requests.get('https://imsdb.com/scripts/12.html').text
).xpath('//td[@class="scrtext"]')


In [35]:
print(
    ''.join(el[0].itertext())
)





  
  12 - Script




CUT FROM BLACK

TITLE: FIN

EXTERIOR - LA - DAY
Fin of red 1957 Chevy Impala convertible driving 
somewhere in the West. A car passes going the other 
way.

TITLE: PLACE: Los Angeles

MUSIC: Shadowy Men On A Shadowy Planet "Good Cop,
Bad Cop."

EXTERIOR - LA - DAY
POV driving on freeway in rain.

EXTERIOR - LA - DAY
People with umbrellas standing on corner.

EXTERIOR - LA - DAY
People with umbrella walking over bridge.

EXTERIOR - LA - DAY
People walk across downtown intersection with 
umbrellas, bus in background.

EXTERIOR - LA - DAY
Rain and shadow on pavement.

EXTERIOR - LA - DAY
LA river wide, medium, close. We see graffiti 
swamped by high water.

EXTERIOR - LA - DAY
Large man walking with umbrella.

WE HEAR: THUNDER 

MUSIC STOPS

			FIRST WOMAN
	I've been here one year and I've lived 
	through an earthquake, fires, floods...

			SECOND WOMAN
	The rains...it just keeps coming...the 					
	floods...the hillsides are coming down... 
	you can't get to and 